In [3]:
import importlib
import dataset
importlib.reload(dataset)
from dataset import extract_matrix_users_movies_ratings, extract_users_movies_ratings_lists

In [4]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import random
from libreco.data import split_by_ratio_chrono, DatasetPure
from libreco.algorithms import ALS
from sklearn.metrics import mean_squared_error

Instructions for updating:
non-resource variables are not supported in the long term


In [5]:
def save_predictions_from_pandas(res_path, predictions, index_pd):
    index_pd = index_pd.astype({'Prediction': 'float'})
    index_pd.iloc[:, 1] = predictions
    index_pd.to_csv(res_path, index=False, float_format='%.3f')

In [6]:
def save_predictions(res_path, predictions):
    test_pd = pd.read_csv('../data/sampleSubmission.csv')
    test_pd = test_pd.astype({'Prediction': 'float'})
    test_pd.iloc[:, 1] = predictions
    test_pd.to_csv(res_path, index=False, float_format='%.3f')

In [7]:
RANDOM_STATE = 58

test_pd = pd.read_csv('../data/sampleSubmission.csv')
users_test, movies_test, _ = extract_users_movies_ratings_lists(test_pd)

In [18]:
scores = []

for i in range(5):
    train_pd = pd.read_csv(f'../data_val_train_kfold/partition_{i}_train.csv')
    val_pd = pd.read_csv(f'../data_val_train_kfold/partition_{i}_val.csv')
    users_train, movies_train, ratings_train = extract_users_movies_ratings_lists(train_pd)
    users_val, movies_val, ratings_val = extract_users_movies_ratings_lists(val_pd)
    train = pd.DataFrame({'user': users_train, 'item': movies_train, 'label': ratings_train})
    val = pd.DataFrame({'user': users_val, 'item': movies_val, 'label': ratings_val})
    train_data, data_info = DatasetPure.build_trainset(train)
    eval_data = DatasetPure.build_evalset(val)
    
    score = eval_ensemble(i, train_data, data_info, users_val, movies_val, ratings_val, users_test, movies_test)
    print(score)
    scores.append(score)

Training start time: 2022-07-31 10:47:52
0.9914511306808664
Training start time: 2022-07-31 10:47:56
0.994756337328748
Training start time: 2022-07-31 10:48:01
0.9971823159714058
Training start time: 2022-07-31 10:48:05
0.9954252164490369
Training start time: 2022-07-31 10:48:09
0.9983482065278693


In [15]:
#als ensemble
np.array(scores).mean(), np.array(scores).std()

(0.9791652918978627, 0.0023202357098710694)

In [19]:
# als emb size 4 reg 0.1
np.array(scores).mean(), np.array(scores).std()

(0.9954326413915853, 0.002361006186096758)

In [17]:
def eval_ensemble(split, train_data, data_info, users_val, movies_val, ratings_val, users_test, movies_test):
    emb_sizes = [1,2,3,4,5,6,8,10,12,15]
    regs = [0.01, 0.05, 0.1, 0.2, 0.3, 0.5, 1]
    
    david_val = np.zeros((len(emb_sizes) * len(regs), len(users_val)))
    david_test = np.zeros((len(emb_sizes) * len(regs), len(users_test)))

    for i, emb_size in enumerate(emb_sizes):
        for j, reg in enumerate(regs):
            giorgio = ALS(task="rating", data_info=data_info, embed_size=emb_size, n_epochs=5,
                        reg=reg, seed=42)
            giorgio.fit(train_data, verbose=0, use_cg=False, n_threads=8, metrics=["rmse", "mae", "r2"])
            david_val[len(regs)*i+j] = np.array(giorgio.predict(user=users_val, item=movies_val))
            david_test[len(regs)*i+j] = np.array(giorgio.predict(user=users_test, item=movies_test))

    yhat_val = david_val.mean(axis=0)
    yhat_test = david_test.mean(axis=0)
    save_predictions_from_pandas(f'../results_ensemble/ALS/ALS_split_{split}_val_results.csv', yhat_val, val_pd)
    save_predictions(f'../results_ensemble/ALS/ALS_split_{split}_test_results.csv', yhat_test)
    
    return mean_squared_error(yhat_val, ratings_val, squared=False)

In [38]:
# --------- generate submission -------
emb_sizes = [1,2,3,4,5,6,8,10,12,15]
regs = [0.01, 0.05, 0.1, 0.2, 0.3, 0.5, 1]
david = np.zeros((len(emb_sizes) * len(regs), len(users_test)))

train_pd = pd.read_csv(f'../data/data_train.csv')
users_train, movies_train, ratings_train = extract_users_movies_ratings_lists(train_pd)
train = pd.DataFrame({'user': users_train, 'item': movies_train, 'label': ratings_train})
train_data, data_info = DatasetPure.build_trainset(train)


for i, emb_size in enumerate(emb_sizes):
    for j, reg in enumerate(regs):
        giorgio = ALS(task="rating", data_info=data_info, embed_size=emb_size, n_epochs=5,
                    reg=reg, seed=42)
        giorgio.fit(train_data, verbose=2, use_cg=False, n_threads=8, metrics=["rmse", "mae", "r2"])
        david[len(regs)*i+j] = np.array(giorgio.predict(user=users_test, item=movies_test))

yhat = david.mean(axis=0)
save_predictions(f'../results_ensemble/ALS/ALS-predictedSubmission.csv', yhat)

Training start time: 2022-07-30 12:56:17
Epoch 1 elapsed: 0.022s
Epoch 2 elapsed: 0.022s
Epoch 3 elapsed: 0.021s
Epoch 4 elapsed: 0.021s
Epoch 5 elapsed: 0.021s
Training start time: 2022-07-30 12:56:17
Epoch 1 elapsed: 0.021s
Epoch 2 elapsed: 0.022s
Epoch 3 elapsed: 0.021s
Epoch 4 elapsed: 0.022s
Epoch 5 elapsed: 0.021s
Training start time: 2022-07-30 12:56:18
Epoch 1 elapsed: 0.022s
Epoch 2 elapsed: 0.035s
Epoch 3 elapsed: 0.022s
Epoch 4 elapsed: 0.022s
Epoch 5 elapsed: 0.022s
Training start time: 2022-07-30 12:56:19
Epoch 1 elapsed: 0.022s
Epoch 2 elapsed: 0.022s
Epoch 3 elapsed: 0.022s
Epoch 4 elapsed: 0.022s
Epoch 5 elapsed: 0.022s
Training start time: 2022-07-30 12:56:20
Epoch 1 elapsed: 0.022s
Epoch 2 elapsed: 0.021s
Epoch 3 elapsed: 0.022s
Epoch 4 elapsed: 0.021s
Epoch 5 elapsed: 0.022s
Training start time: 2022-07-30 12:56:21
Epoch 1 elapsed: 0.021s
Epoch 2 elapsed: 0.022s
Epoch 3 elapsed: 0.022s
Epoch 4 elapsed: 0.022s
Epoch 5 elapsed: 0.022s
Training start time: 2022-07-30 12

Epoch 5 elapsed: 0.039s
Training start time: 2022-07-30 12:56:39
Epoch 1 elapsed: 0.038s
Epoch 2 elapsed: 0.038s
Epoch 3 elapsed: 0.039s
Epoch 4 elapsed: 0.038s
Epoch 5 elapsed: 0.038s
Training start time: 2022-07-30 12:56:40
Epoch 1 elapsed: 0.040s
Epoch 2 elapsed: 0.039s
Epoch 3 elapsed: 0.040s
Epoch 4 elapsed: 0.038s
Epoch 5 elapsed: 0.043s
Training start time: 2022-07-30 12:56:41
Epoch 1 elapsed: 0.061s
Epoch 2 elapsed: 0.048s
Epoch 3 elapsed: 0.047s
Epoch 4 elapsed: 0.050s
Epoch 5 elapsed: 0.047s
Training start time: 2022-07-30 12:56:42
Epoch 1 elapsed: 0.047s
Epoch 2 elapsed: 0.048s
Epoch 3 elapsed: 0.048s
Epoch 4 elapsed: 0.047s
Epoch 5 elapsed: 0.051s
Training start time: 2022-07-30 12:56:43
Epoch 1 elapsed: 0.047s
Epoch 2 elapsed: 0.048s
Epoch 3 elapsed: 0.047s
Epoch 4 elapsed: 0.047s
Epoch 5 elapsed: 0.051s
Training start time: 2022-07-30 12:56:44
Epoch 1 elapsed: 0.047s
Epoch 2 elapsed: 0.048s
Epoch 3 elapsed: 0.047s
Epoch 4 elapsed: 0.047s
Epoch 5 elapsed: 0.049s
Training s

Epoch 2 elapsed: 0.111s
Epoch 3 elapsed: 0.110s
Epoch 4 elapsed: 0.111s
Epoch 5 elapsed: 0.107s
Training start time: 2022-07-30 12:57:08
Epoch 1 elapsed: 0.109s
Epoch 2 elapsed: 0.125s
Epoch 3 elapsed: 0.108s
Epoch 4 elapsed: 0.110s
Epoch 5 elapsed: 0.112s
Training start time: 2022-07-30 12:57:09
Epoch 1 elapsed: 0.107s
Epoch 2 elapsed: 0.110s
Epoch 3 elapsed: 0.111s
Epoch 4 elapsed: 0.108s
Epoch 5 elapsed: 0.108s
Training start time: 2022-07-30 12:57:10
Epoch 1 elapsed: 0.106s
Epoch 2 elapsed: 0.109s
Epoch 3 elapsed: 0.107s
Epoch 4 elapsed: 0.108s
Epoch 5 elapsed: 0.108s
Training start time: 2022-07-30 12:57:12
Epoch 1 elapsed: 0.107s
Epoch 2 elapsed: 0.108s
Epoch 3 elapsed: 0.106s
Epoch 4 elapsed: 0.108s
Epoch 5 elapsed: 0.107s
Training start time: 2022-07-30 12:57:13
Epoch 1 elapsed: 0.135s
Epoch 2 elapsed: 0.137s
Epoch 3 elapsed: 0.135s
Epoch 4 elapsed: 0.144s
Epoch 5 elapsed: 0.135s
Training start time: 2022-07-30 12:57:14
Epoch 1 elapsed: 0.136s
Epoch 2 elapsed: 0.137s
Epoch 3 el